In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import gc
print(datetime.datetime.now())

2020-06-29 16:49:28.246028


In [3]:
list_all_quarterly_files=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/all_LR_returned/*.gz")
list_all_quarterly_files=sorted(list_all_quarterly_files,key=lambda x: os.path.basename(x).split("_")[4] if "combine" in x else os.path.basename(x).split("_")[7].replace("-",""))
list_all_quarterly_files

['/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/all_LR_returned/CL_BigLots_allrewards_combined1011_20181026_232208_0000.psv.gz',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/all_LR_returned/CL_BigLots_allrewards_combined0908_20181029_171317_0000.psv.gz',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/all_LR_returned/CL_BigLots_BL_mapping_file_2018Q4_JL_2019-02-01_20190207_025237_0000.psv.gz',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/all_LR_returned/CL_BigLots_BL_mapping_file_2019Q1_JL_2019-05-14_20190516_111931_0000.psv.gz',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/all_LR_returned/CL_BigLots_BL_mapping_file_2019Q2_JL_2019-09-23_20190926_084457_0000.psv.gz',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/all_LR_returned/CL_BigLots_BL_mapping_file_2019Q3_JL_2019-11-20_20191125_232935_0000.psv.gz',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/all_LR_returned/CL_BigLots_BL_mapping_file_2019Q4_JL_2020-02-14_20200215_0207

In [4]:
df_total_mapping=pd.DataFrame(columns=['Customer_Link','customer_id_hashed','date_up_to'])
print("Start",datetime.datetime.now())
i=0
for file in list_all_quarterly_files:
    i+=1
    basename=os.path.basename(file)
    if "combined" in file:
        date_up_to=basename.split("_")[4]
    else:
        date_up_to=basename.split("_")[7].replace("-","")
    df=pd.read_table(file,dtype=str,sep="|",compression="gzip",usecols=['Customer_Link','customer_id_hashed'])
    df=df[df['Customer_Link']!="UNMATCHED"].drop_duplicates("Customer_Link")
    df['date_up_to']=date_up_to
    df_pre=df_total_mapping.drop_duplicates("Customer_Link") # as the high date in the cum mapping
    df_pre['latest']=1 # label as the latest avaibale
    df_total_mapping=pd.merge(df_total_mapping,df_pre,on=["Customer_Link","customer_id_hashed","date_up_to"],how="left")
    df_total_mapping=df_total_mapping[pd.isnull(df_total_mapping['latest'])] # remove the latest and kept previous only if different
    del df_pre['latest']
    del df_total_mapping['latest']
    df_2_consecutive=df.append(df_pre) # new on the top, old below
    
    df_ids_count_by_idl=df_2_consecutive.groupby("Customer_Link")["customer_id_hashed"].nunique().to_frame().reset_index()
    df_ids_count_by_idl=df_ids_count_by_idl.rename(columns={"customer_id_hashed":"BL_id_count"})
    df_ids_count_by_idl_2=df_ids_count_by_idl[df_ids_count_by_idl['BL_id_count']>1] # the new diffrent from the old high date
    df_ids_count_by_idl_1=df_ids_count_by_idl[df_ids_count_by_idl['BL_id_count']==1]# the new same as the old high date
    # 1 and 2 should be mutually exclusive
    df_1=pd.merge(df_2_consecutive,df_ids_count_by_idl_1,on="Customer_Link",how="inner").drop_duplicates("Customer_Link") # keep the new only
    df_2=pd.merge(df_2_consecutive,df_ids_count_by_idl_2,on="Customer_Link",how="inner") # keep both
    
    del df_1['BL_id_count']
    del df_2['BL_id_count']
    
    print(df_1.columns.tolist())
    print(df_2.columns.tolist())
    
    df_total_mapping=pd.concat([df_1,df_2,df_total_mapping])
    
    print(i,datetime.datetime.now())
print("Done",datetime.datetime.now())
del df_1
del df_2
del df_2_consecutive
del df_ids_count_by_idl
del df_ids_count_by_idl_1
del df_ids_count_by_idl_2
del df
gc.collect()
df_total_mapping=df_total_mapping.sort_values(['Customer_Link','date_up_to'],ascending=[True,False])
gc.collect()
df_total_mapping.shape
    

Start 2020-06-29 16:49:55.551758
['Customer_Link', 'customer_id_hashed', 'date_up_to']
['Customer_Link', 'customer_id_hashed', 'date_up_to']
1 2020-06-29 16:50:40.216910
['Customer_Link', 'customer_id_hashed', 'date_up_to']
['Customer_Link', 'customer_id_hashed', 'date_up_to']
2 2020-06-29 17:00:48.587629


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


['Customer_Link', 'customer_id_hashed', 'date_up_to']
['Customer_Link', 'customer_id_hashed', 'date_up_to']
3 2020-06-29 17:25:03.789837
['Customer_Link', 'customer_id_hashed', 'date_up_to']
['Customer_Link', 'customer_id_hashed', 'date_up_to']
4 2020-06-29 17:52:22.577559
['Customer_Link', 'customer_id_hashed', 'date_up_to']
['Customer_Link', 'customer_id_hashed', 'date_up_to']
5 2020-06-29 18:20:21.871722
['Customer_Link', 'customer_id_hashed', 'date_up_to']
['Customer_Link', 'customer_id_hashed', 'date_up_to']
6 2020-06-29 18:51:58.568523
['Customer_Link', 'customer_id_hashed', 'date_up_to']
['Customer_Link', 'customer_id_hashed', 'date_up_to']
7 2020-06-29 19:26:42.645216
['Customer_Link', 'customer_id_hashed', 'date_up_to']
['Customer_Link', 'customer_id_hashed', 'date_up_to']
8 2020-06-29 20:02:17.610844
Done 2020-06-29 20:02:17.612643


(40750089, 3)

In [5]:
df_total_mapping.head(2)

,Customer_Link,customer_id_hashed,date_up_to
20164975,XY1468---077tVclzugrAPADjYKKd1D3ZebCh26abPI1mraTs,c038e1b89f0bf1902d4c365d001b9a19b3a739358d1ee7...,20200513
22051056,XY1468---0pTSu_zfDGheJ-qYQPfvwhCI1e7f_joTNdovjomE,1b5cda9e5ca0af148177d6a317905639cb35d744e0ff2d...,20200513


In [6]:
print("df_total_mapping.shape",df_total_mapping.shape)
print("df_total_mapping['Customer_Link'].nunique()",df_total_mapping['Customer_Link'].nunique())
print("df_total_mapping['customer_id_hashed'].nunique()",df_total_mapping['customer_id_hashed'].nunique())

df_total_mapping.shape (40750089, 3)
df_total_mapping['Customer_Link'].nunique() 39212759
df_total_mapping['customer_id_hashed'].nunique() 34610603


In [7]:
df_total_mapping.to_csv("./df_total_mapping_IDL_BLid_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
print(datetime.datetime.now())

2020-06-29 20:22:43.945720
